# Hi-C解析実習
  
[HiCExplorer](https://hicexplorer.readthedocs.io)パイプラインでHi-C実験データの解析をしてみる。  

## 0. HiCExplorerのインストール








In [2]:
#!conda install hicexplorer -c bioconda -c conda-forge

## 1. データのダウンロード
扱うデータは、[Wibke Schwarzer, Nezar Abdennur, Anton Goloborodko, et al. Nature (2017)](https://www.nature.com/articles/nature24281)（以下、Schwarzer2017）。  
この論文では、コヒーシンの染色体高次構造形成への影響を調べるため、コヒーシンをクロマチンにロードする役割を持つNIPBLを欠損させたマウスの肝細胞でHi-Cを行なっている。   
コヒーシンが染色体から外れてしまっているので、構造はだいぶ異なるはず。   
この実習では、（ほんとはfloxと比較すべきだけど）野生型とΔNipblで染色体構造がどのように異なるか、それと、論文の図を再現できるかを、公共データベースに公開されているシーケンスデータから自分で解析して確かめてみる。   


<img src="data/images/experimental_design.png" width=200 align="left"><br>
　Schwarzer2017, Fig.1より

まずはデータのダウンロード。  
WTのサンプルである**SRR5168118**, ΔNipblサンプルである**SRR5168150**それぞれについて、以下のようにしてEMBL-EBI ENAからデータを取ってくる。  
（fastq-dumpでもいいが個人的にはENAの方がなんか安心感があって好き）   
  
実際はそれぞれのサンプル数億本のリード数（x 複数レプリケイト）だが、実習の時間的環境的都合上、ここで扱うのはそのほんの一部（SRR5168118: 18,063,305本、SRR5168150: 17,105,434本）のみ。  
Hi-C解析はリード数が結果の解像度に直結するので、数千万本程度だと、かなり粗い結果しか得られないことに注意。

In [7]:
%%bash
<< SKIP # 実習ではダウンロード済み
wget -O SRR5168118_1.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR516/008/SRR5168118/SRR5168118_1.fastq.gz
wget -O SRR5168118_2.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR516/008/SRR5168118/SRR5168118_2.fastq.gz

wget -O SRR5168150_1.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR516/000/SRR5168150/SRR5168150_1.fastq.gz
wget -O SRR5168150_2.fastq.gz ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR516/000/SRR5168150/SRR5168150_2.fastq.gz
SKIP

## 2. マッピング
マウスのデータなので、mm10（[GRCm38](https://www.ncbi.nlm.nih.gov/assembly/GCF_000001635.20/)）ゲノム配列にマッピングする。  
他のXX-seqデータと違って、Hi-C解析に特有の注意点がいくつかある。  
+ マッパーによっては、ペアエンドファイルを入力するとインサートサイズを適当に仮定してすばやく検索するヒューリスティックが起動してしまう。Hi-Cデータの場合ペアはゲノム上ですごく遠い場所（あるいは別の染色体）に由来するかもしれない。なので、R1とR2は個別にマッピングする。
+ ローカルアラインメントをする。グローバルアラインメントはダメ。Hi-Cリードは本質的にligation productで、chimeric sequenceなため。
+ アラインメントのパラメータを適切に調整する。HiCExplorerでは、BWAマッピングの場合は以下が推奨されている。
    - -A1 マッチスコア。1に設定（デフォルト）
    - -B4 ミスマッチペナルティ。4に設定（デフォルト）
    - -O6 ギャップ開始ペナルティ。6に設定（デフォルト）
    - -E50 ギャップ伸長ペナルティ。デフォルトは1だが、ずっと大きく50にする。
    - -L0 クリッピングペナルティ。5'側あるいは3'側にSmith-Watermanのスコア計算を伸ばしていって、ベストスコアのところで止めるか端っこまで到達させるかを調整するペナルティ。これが大きいとグローバルアラインメントになりやすい。デフォルトは5だが、これをゼロに設定。
    - 以上の設定にすればローカルアラインメントが得られやすくなる。  

In [6]:
%%bash
<< SKIP #マッピングも時間の関係で省略
datadir=data
reffile=mm10.fa

bwa index ${reffile}

for srr_id in "SRR5168118" "SRR5168150"
do
    echo $srr_id
    infile_1=${datadir}/${srr_id}_1.fastq.gz
    infile_2=${datadir}/${srr_id}_2.fastq.gz
    bwa mem -A1 -B4  -E50 -L0 $reffile \
        $infile_1 2>> ${srr_id}.r1.log | samtools view -Shb - > ${datadir}/${srr_id}_R1.bam
    bwa mem -A1 -B4  -E50 -L0 $reffile \
        $infile_2 2>> ${srr_id}.r2.log | samtools view -Shb - > ${datadir}/${srr_id}_R2.bam
done
SKIP

## 3. リードのフィルタリングとコンタクトマップの構築

マッピングされたペアのすべたがHi-Cのコンタクト情報として使えるわけではない。  
以下のように様々な理由で「コンタクト」を反映していないペアが存在する。  
不適切なペアは、マッピングのパターン、すなわちマッピングされたStrand（= orientation）や、実験に使った制限酵素の認識配列の位置との関係などから判断することができる。  

<img src="data/images/pair_mapping_patterns.png" width=600 align="left"><br>
　Imakaev, Maxim, et al. "Iterative correction of Hi-C data reveals hallmarks of chromosome organization.” Nature methods 9.10 (2012): 999-1003.より

そこで、R1.bam, R2.bamから、valid-pairの情報のみを抽出し、さらに指定した解像度（Binサイズ）でコンタクトマップのデータとしてマッピングの位置情報を集計する。  
それを一気にやってくれるのが HiCExplorer の ```hicBuildMatrix``` コマンド。

In [20]:
%%bash
<< SKIP #このプロセスも、数ギガバイトのbamファイルをダウンロードさせるのがきつかったのでスキップ。実際に計算するのは次から。
outdir=data/build_matrix
for srr_id in "SRR5168118" "SRR5168150"
do
    echo $srr_id
    infile_1=data/${srr_id}_R1.bam
    infile_2=data/${srr_id}_R2.bam

    qcdir=${outdir}/${srr_id}_qc
    outfile=${outdir}/${srr_id}_100kb.h5
    hicBuildMatrix --samFiles ${infile_1} ${infile_2} \ # bamファイル、R1とR2を指定。
        --binSize 100000 \ # コンタクトマップのBinサイズ。この実習ではリード数がめっちゃ少ないので100kbpの粗い解像度。
                                      #  制限酵素認識配列の位置情報をbedファイルで与えることで、制限断片の解像度（不定Binサイズ）のマップを作ることもできる。
        --restrictionSequence AAGCTT  \　# 制限酵素認識配列。論文読むとHindIII処理していたので、その認識配列。実験によって異なる。
        --danglingSequence AGCT \ # 制限消化したときの突出末端の配列。HindIIIの場合はAGCT。
        --outFileName ${outfile} \ # 結果のコンタクトマップ。HDF5フォーマット。
        --QCfolder ${qcdir_name} \ # クオリティ情報をまとめて出してくれる。そのディレクトリを指定。
        --threads 32 # 計算環境に合わせて適切に...
done
SKIP

設定したQCディレクトリにさまざまな統計情報を出力してくれている。

In [24]:
!ls -l ./data/build_matrix/SRR5168118_qc/

total 712
-rw-r--r--@ 1 higashi  staff   62554 11 10 05:20 distance.png
-rw-r--r--@ 1 higashi  staff  112230 11 10 05:21 pairs_discarded.png
-rw-r--r--@ 1 higashi  staff   59932 11 10 05:22 pairs_sequenced.png
-rw-r--r--@ 1 higashi  staff   42543 11 10 05:22 read_orientation.png
-rw-r--r--@ 1 higashi  staff   76873 11 10 05:23 unmappable_and_non_unique.png


<img src="data/build_matrix/SRR5168118_qc/pairs_sequenced.png" width=600 align="left"><br>
　これはvalid-pairの数（コンタクトマップの計算に実際に使われるペア）   
　もともと1,750万ペアほどあったのが、800万ペアほどに減っている。

<img src="data/build_matrix/SRR5168118_qc/unmappable_and_non_unique.png" width=600 align="left"><br>
　全部のペアに対する、マッピングのクオリティなどで基準を通過したペアの割合。

<img src="data/build_matrix/SRR5168118_qc/pairs_discarded.png" width=600 align="left"><br>
　mappable pairsのうち、「コンタクト」を反映しないダメなペアの割合。これらは除去される。
 
 


+ dangling ends: 制限消化された部位からはじまるリード。ligationに失敗してるやつ。
+ duplicated pairs: 同じ位置にマッピングされるペア。PCR増幅の疑い
+ same fragment: 800bp以内の距離で →　← の方向でマッピングされるペア。同じ制限断片に由来
+ self circle: 25kbp以内の距離で ←　→ の方向でマッピングされるペア。自己環状化断片の疑い。
+ self ligation: same fragmentと同じだが、内部に制限酵素認識サイトあり。同じ位置で開いて閉じたやつ。

<img src="data/build_matrix/SRR5168118_qc/distance.png" width=600 align="left"><br>
最終的にコンタクトマップに集計された「コンタクト」の位置関係。  
この図はQCとして重要な情報を含む。  
Hi-C実験、うまくいってないと、異常に高い "inter-chromosomal contact" （異なる染色体間のコンタクト）割合が観測されることがある。（構造が壊れて染色体間の接触がランダムに観測される）    
このサンプルの場合は、25%程度なのでまあまあ普通のクオリティ。  
　
 

<img src="data/build_matrix/SRR5168118_qc/read_orientation.png" width=600 align="left"><br>
　最終的にコンタクトマップに集計されたペアリードのマッピングの方向。  
　→←、←→、←←、→→の4パターン。  
　ちゃんと網羅的に接触がとれてるなら4パターンの割合に偏りはないはず。  
　どれかが極端に多い・少ない場合は注意が必要。  


## 4. コンタクトマップの正規化

RNA-seqやメタゲノムなど「一次元データ」の正規化とは異なり、Hi-C解析の場合は二次元データに適した正規化手法を適用する必要がある。  
よく使われるのは ICE法（[Imakaev, Maxim, et al. "Iterative correction of Hi-C data reveals hallmarks of chromosome organization." Nature methods 9.10 (2012): 999.](https://www.nature.com/articles/nmeth.2148)）で、matrix balancingと呼ばれる数学的手法の一種。  
HiCExplorerにもICE法のfuncitonがあるが、ノイズが爆発することを防ぐために適切に閾値を設定したりちょっと面倒。  
より高速なmatrix balancingの手法として、[Knight-Ruiz法](https://pdfs.semanticscholar.org/4f5d/26b58c3ad6697bada5553496c62fc2a91295.pdf)もよく使われる。  
```hicCorrectMatrix```コマンドでは、```--correctionMethod KR```と指定することで、Knight-Ruiz法による正規化を実行することができる。

In [1]:
%%bash
datadir=data/build_matrix
outdir=data/correct_matrix
for srr_id in "SRR5168118" "SRR5168150"
do
    echo $srr_id
    matrixfile=${datadir}/${srr_id}_100kb.h5
    outfile=${outdir}/${srr_id}_100kb_corrected.h5
    hicCorrectMatrix correct \
        --correctionMethod KR \
        --matrix ${matrixfile} \
        --outFileName ${outfile}
done

SRR5168118
normalisation factor is 0.0425437
SRR5168150
normalisation factor is 0.045753


INFO:hicexplorer.hicCorrectMatrix:matrix contains 8891709 data points. Sparsity 0.012.
INFO:hicexplorer.hicCorrectMatrix:matrix contains 8570869 data points. Sparsity 0.012.


## 5. コンタクトマップの描画とA/Bコンパートメント計算